### 데이터 작업하기

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
# 공개 데이터셋에서 학습 데이터 내려받기
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

# 공개 데이터셋에서 테스트 데이터 내려받기
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

In [3]:
print(training_data, '\n')
print(test_data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor() 

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [4]:
batch_size = 64

# DataLoader 생성
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    print("Example of y: ", y)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64
Example of y:  tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1,
        2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5])


### 모델 만들기

- \_\_init\_\_ 함수 : 신경망 layer 정의
- forward 함수 : 신경망에 데이터를 어떻게 전달할지 지정

In [5]:
# 학습에 사용할 CPU나 GPU 장치 지정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# 모델 정의하기
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 모델 매개변수 최적화하기

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [7]:
# len(train_dataloader) : 938 (= math.ceil(60000 / 64))
# len(train_dataloader.dataset) : 60000
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)               # number of train_dataloader samples : 60000
    for batch, (X, y) in enumerate(dataloader):  # range(938)
        X, y = X.to(device), y.to(device)
        
        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)               # len(X) : 64
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
# len(test_dataloader) : 157 (= math.ceil(10000 / 64))
# len(test_dataloader.dataset) : 10000
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)              # number of test_dataloader samples : 10000
    num_batches = len(dataloader)               # 157
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:                 # range(157)
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()   # batch 내에 있는 모든 샘플들의 정답 일치 여부 확인
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1}\n---------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------------------------
loss: 2.299303  [    0/60000]
loss: 2.288324  [ 6400/60000]
loss: 2.269500  [12800/60000]
loss: 2.272497  [19200/60000]
loss: 2.244609  [25600/60000]
loss: 2.226052  [32000/60000]
loss: 2.227932  [38400/60000]
loss: 2.197902  [44800/60000]
loss: 2.207786  [51200/60000]
loss: 2.172077  [57600/60000]
Test Error: 
 Accuracy: 41.6%, Avg loss: 2.164954 

Epoch 2
---------------------------
loss: 2.174172  [    0/60000]
loss: 2.161851  [ 6400/60000]
loss: 2.108220  [12800/60000]
loss: 2.131803  [19200/60000]
loss: 2.070730  [25600/60000]
loss: 2.023987  [32000/60000]
loss: 2.042817  [38400/60000]
loss: 1.971409  [44800/60000]
loss: 1.988590  [51200/60000]
loss: 1.909857  [57600/60000]
Test Error: 
 Accuracy: 55.6%, Avg loss: 1.906232 

Epoch 3
---------------------------
loss: 1.939901  [    0/60000]
loss: 1.905677  [ 6400/60000]
loss: 1.794354  [12800/60000]
loss: 1.838351  [19200/60000]
loss: 1.717839  [25600/60000]
loss: 1.674184  [32000/60000]
loss: 1.

### 모델 저장하기

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### 모델 불러오기

In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
